# Imports

In [381]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# Load Data

In [382]:
data = pd.read_csv('data/DC_Properties.csv')

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Exploration

In [383]:
data.shape # we have 49 columns and 158957 rows

(158957, 49)

In [384]:
missing_values = data.isnull().sum()

print("Row Count: " + str(data.shape[0]))
print()
print(missing_values)

Row Count: 158957

Unnamed: 0                 0
BATHRM                     0
HF_BATHRM                  0
HEAT                       0
AC                         0
NUM_UNITS              52261
ROOMS                      0
BEDRM                      0
AYB                      271
YR_RMDL                78029
EYB                        0
STORIES                52305
SALEDATE               26770
PRICE                  60741
QUALIFIED                  0
SALE_NUM                   0
GBA                    52261
BLDG_NUM                   0
STYLE                  52261
STRUCT                 52261
GRADE                  52261
CNDTN                  52261
EXTWALL                52261
ROOF                   52261
INTWALL                52261
KITCHENS               52262
FIREPLACES                 0
USECODE                    0
LANDAREA                   0
GIS_LAST_MOD_DTTM          0
SOURCE                     0
CMPLX_NUM             106696
LIVING_GBA            106696
FULLADDRESS            5

In [385]:
data[data["STYLE"].notnull()].shape # 52.261 rows don't have a value for the "STYLE" column

(106696, 49)

In [386]:
data[data["STRUCT"].notnull() | data["STYLE"].notnull() | data["GRADE"].notnull() | data["CNDTN"].notnull() | data["EXTWALL"].notnull() | data["ROOF"].notnull() | data["INTWALL"].notnull()].shape # the same rows don't have a value for: "STRUCT", "GRADE", "CNDTN", "EXTWALL", "ROOF", "INTWALL"

(106696, 49)

In [387]:
# command copied form data cleaning
data_dropped_rows = data[data["STRUCT"].notnull() | data["STYLE"].notnull() | data["GRADE"].notnull() | data["CNDTN"].notnull() | data["EXTWALL"].notnull() | data["ROOF"].notnull() | data["INTWALL"].notnull()]

In [388]:
missing_values = data_dropped_rows.isnull().sum()

print("Row Count: " + str(data_dropped_rows.shape[0]))
print()
print(missing_values)

Row Count: 106696

Unnamed: 0                 0
BATHRM                     0
HF_BATHRM                  0
HEAT                       0
AC                         0
NUM_UNITS                  0
ROOMS                      0
BEDRM                      0
AYB                      241
YR_RMDL                57417
EYB                        0
STORIES                   44
SALEDATE               22513
PRICE                  48796
QUALIFIED                  0
SALE_NUM                   0
GBA                        0
BLDG_NUM                   0
STYLE                      0
STRUCT                     0
GRADE                      0
CNDTN                      0
EXTWALL                    0
ROOF                       0
INTWALL                    0
KITCHENS                   1
FIREPLACES                 0
USECODE                    0
LANDAREA                   0
GIS_LAST_MOD_DTTM          0
SOURCE                     0
CMPLX_NUM             106696
LIVING_GBA            106696
FULLADDRESS             

1) no row has a value for the cmplx_num and the living_gba => useless
2) there are many missing values for the sale_date and the yr_rmdl => maybe they aren't missing, it could be the case, that these houses were never remodeled and/ or sold before, in the following we will probably handle them this way.

There is a column called "Unnamed: 0", lets see what's in there.

In [389]:
data["Unnamed: 0"]

0              0
1              1
2              2
3              3
4              4
5              5
6              6
7              7
8              8
9              9
10            10
11            11
12            12
13            13
14            14
15            15
16            16
17            17
18            18
19            19
20            20
21            21
22            22
23            23
24            24
25            25
26            26
27            27
28            28
29            29
           ...  
158927    158927
158928    158928
158929    158929
158930    158930
158931    158931
158932    158932
158933    158933
158934    158934
158935    158935
158936    158936
158937    158937
158938    158938
158939    158939
158940    158940
158941    158941
158942    158942
158943    158943
158944    158944
158945    158945
158946    158946
158947    158947
158948    158948
158949    158949
158950    158950
158951    158951
158952    158952
158953    158953
158954    1589

It's just the same as the index (probably the unique id column).

In [390]:
data["STATE"].unique() 

array(['DC', nan], dtype=object)

In [391]:
data["CITY"].unique()

array(['WASHINGTON', nan], dtype=object)

Since our dataset is only about Washington DC these columns habe no use.

In [392]:
data["ZIPCODE"].unique()

array([ 20009.,  20037.,  20036.,  20005.,  20052.,  20008.,  20001.,
        20024.,  20006.,  20003.,  20002.,  20007.,  20016.,  20015.,
        20011.,  20010.,  20012.,  20017.,  20018.,  20019.,  20020.,
        20032.,     nan,  20392.,  20004.])

The ZIPCODE on the otherside contains some value

In [393]:
data["FULLADDRESS"].unique()

array(['1748 SWANN STREET NW', '1746 SWANN STREET NW',
       '1744 SWANN STREET NW', ..., '131 JOLIET STREET SW',
       '135 JOLIET STREET SW', '139 JOLIET STREET SW'], dtype=object)

In [394]:
data["FULLADDRESS"].unique().shape

(105979,)

In [395]:
data.shape[0] - data["FULLADDRESS"].isnull().sum()

106040

The column "FULLADDRESS" has only unique text values, the computer can't process them. Either you split them up (street and house number or just leave them out because the streetnames aren't that useful either)

In [396]:
data["NATIONALGRID"].unique()

array(['18S UJ 23061 09289', '18S UJ 23067 09289', '18S UJ 23074 09289',
       ..., '18S UH 25815 98879', '18S UH 25807 98865',
       '18S UH 25804 98859'], dtype=object)

In [397]:
data["NATIONALGRID"].unique().shape

(105950,)

In [398]:
data.shape[0] - data["NATIONALGRID"].isnull().sum()

106051

The column "NATIONALGRID" has the same problem, as the one above.

In [399]:
data["GIS_LAST_MOD_DTTM"].unique()

array(['2018-07-22 18:01:43', '2018-07-22 18:01:38'], dtype=object)

This row says when the row of the dataset was last modified, it's useless for us.

In [400]:
data["SALEDATE"].unique()

array(['2003-11-25 00:00:00', '2000-08-17 00:00:00', '2016-06-21 00:00:00',
       ..., '1995-02-16 00:00:00', '1994-02-14 00:00:00',
       '1993-08-06 00:00:00'], dtype=object)

This row contains the date, the building was sold. The way it is saved is not good to compute, we will convert it later.

In [401]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.ticklabel_format(style = 'plain')

data.boxplot(column=["PRICE"])
plt.show()

In [402]:
data.sort_values(by="PRICE")["PRICE"]

132821       1.0
148694       1.0
152131       1.0
81694        1.0
106683       1.0
18568       10.0
74419       10.0
81594       10.0
51658       10.0
48414      250.0
14041      250.0
55048      250.0
102782     250.0
43281      250.0
7396       250.0
42069      250.0
50971      250.0
49321      250.0
93541      250.0
94173      250.0
54909      250.0
49443      250.0
84165      250.0
49470      500.0
60597      500.0
5516       500.0
54229      500.0
3330       936.0
153470    1000.0
100485    1000.0
           ...  
158845       NaN
158847       NaN
158848       NaN
158849       NaN
158850       NaN
158852       NaN
158855       NaN
158856       NaN
158857       NaN
158859       NaN
158860       NaN
158863       NaN
158865       NaN
158866       NaN
158870       NaN
158872       NaN
158879       NaN
158883       NaN
158889       NaN
158913       NaN
158917       NaN
158922       NaN
158923       NaN
158925       NaN
158927       NaN
158937       NaN
158940       NaN
158941       N

The price of some buildings is suspicous, we have some verry verry high prices (140.000.000$) and some realy low prices (1$) 

In [403]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.ticklabel_format(style = 'plain')

data.loc[(data["PRICE"] <= 25000000) & (data["PRICE"] >= 60000)].boxplot(column=["PRICE"])
plt.show()

In [404]:
data["AC"].unique()

array(['Y', 'N', '0'], dtype=object)

A zero in there is strange value it stands for NULL, therefore we should replace it with NULL instead of 0.

In [405]:
data["BLDG_NUM"].unique()

array([1, 2, 3, 4, 5], dtype=int64)

We haven't found anything about this column in the documentation and since it has only two possible values, it seems kind of useless.